# Soil moisture mapping module

This module runs PYSMM, a python package for creating soil moisture maps. The module can be used to create a time series of soil moisture maps. The estimation of soil moisture is based on a Support-Vector-Regression machine learning approach, training the model based on in-situ data from the International Soil Moisture Network (ISMN) on cross-calibration between Sentinel-1 and Global Land Data Assimilation System (GLDAS). More information about PYSMM can be found on https://pysmm.readthedocs.io/en/latest/


The PYSMM package was developed by Felix Greifeneder with modification from the SEPAL team. 


In [ ]:
#set user libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output, Image
from ipywidgets import HBox, Label, interact
import subprocess
import getpass
import os
import sys
import ee
import itertools
from IPython.core.display import HTML 
ee.Initialize()

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



# Introduction

In [ ]:
user = getpass.getuser()
print(f"Welcome {user}!, run this application to create soil moisture maps from Sentinel 1 imagery.")

# Set user variables

## Define the area of interest
### Provide the link to a Google Earth Engine asset and define the column name and value to select

In [ ]:
aoi =  widgets.Text(
#     value='users/yelenafinegold/107_phu_merged',
    placeholder='Enter your asset ID here',
    disabled=False
)
ui1 = HBox([Label('Google Earth Engine asset ID: '),aoi])
display(ui1)

In [ ]:
aoi_ee = ee.FeatureCollection(aoi.value)
col_ee = ee.Feature(aoi_ee.first()).propertyNames().getInfo()

column_name =  widgets.Select(
    value='id',
    options=col_ee,
    placeholder='column name from the GEE asset',
    disabled=False
)
ui2 = HBox([Label('Column name: '),column_name])
display(ui2)

In [ ]:
id_values_ee = aoi_ee.distinct(column_name.value).aggregate_array(column_name.value).getInfo()
column_value = widgets.Select(
    options=id_values_ee,
    placeholder='column value from the GEE asset',
    disabled=False
)
ui3 = HBox([Label('Column value: '),column_value])
display(ui3)


In [ ]:
# Modify the Year, Month, and Day parameters as desired
year = widgets.SelectMultiple(
    options=range(2014,2021),
#     value=[2015,2016,2017,2018,2019],
    description='Year',
    disabled=False
)

display(year)

month = widgets.SelectMultiple(
    options=[('January',1),('February',2),('March',3),('April',4),('May',5),
             ('June',6),('July',7),('August',8),('September',9),('October',10),
             ('November',11),('December',12)
    ],
#     value=[4,10],
    description='Month',
    disabled=False
)

display(month)

day = widgets.SelectMultiple(
    options=range(1,32),
#     value=[1],
    description='Day',
    disabled=False
)

display(day)


# Run the PYSMM code for a specified AOI - will iterate over all combinations of Year / Month / Day sequentially.

In [ ]:
study_area = aoi_ee.filterMetadata(column_name.value,'equals', float(column_value.value)).geometry().bounds().coordinates()
coords = study_area.get(0).getInfo()

ll = coords[0]
ur = coords[2]
minlon = ll[0]
minlat = ll[1]
maxlon = ur[0]
maxlat = ur[1]
button = widgets.Button(description='Run SMM')
# button, output, function and linkage
outt = widgets.Output()

year1 = list(year.value)
month1 = list(month.value)
day1 = list(day.value)
attribute_value = [aoi.value,column_value.value]

print(f'Asset: {aoi.value}')
print(f'Column: {column_name.value}')
print(f'Column value: {attribute_value[1]}')
print(f'Years: {year1}' )
print(f'Months: {month1}')
print(f'Days: {day1}' )

def on_button_clicked(b):
    with outt:
        clear_output()
        print('Processing and downloading soil moisture maps. This takes a long time')
        print(f'Using the feature: {column_value.value} and the aoi: "{aoi.value}"')
#         !mv ./nohup.out ./nohup_backup.out
#         py = sys.executable
        %run ~/shared/notebooks/sepal_pysmm/scripts/run_pysmm.py "$year1" "$month1" "$day1" $minlon $minlat $maxlon $maxlat $attribute_value
        print('done!')
        
button.on_click(on_button_clicked)
widgets.VBox([button,outt])


# Post processing
## after the download completes - check your pysmm_downloads in your SEPAL folders


This tool reads all the data in the pysmm_downloads folder and applies the ORFEO tool box grayscale morphological operations on all the soil moisture maps. The grayscale morphological operation is a type of image filter that corrects for noise and missing data in the image. The filtered data is called 'closed'.

After the data is filtered, the time series trend analysis of the soil moisture maps is performed. A linear model is applied to the time series of data and results in 2 outputs. The first output is the slope of the linear model. The slope indicates if the trend in soil moisture is negative or positive. These trends might be related to peatland management practices. The second output of the linear regression model is the p-value which indicates the significance of the model. The p-values range between 0 and 1. The closer the p-value output is to 0, the higher the model significance. If the p-value is high (closer to 0), the higher the probability that there is no linear relationship in the time series of soil moisture data. 

After the processing completes, download the outputs and check them in a GIS environment such as QGIS or ArcGIS. 

In [ ]:
class PathSelector():

    def __init__(self,start_dir,select_file=True):
        self.file        = None 
        self.select_file = select_file
        self.cwd         = start_dir
        self.select      = widgets.SelectMultiple(options=['init'],value=(),rows=10,description='') 
        self.accord      = widgets.Accordion(children=[self.select]) 

        self.accord.selected_index = None # Start closed (showing path only)
        self.refresh('.')
        self.select.observe(self.on_update,'value')

    def on_update(self,change):
        if len(change['new']) > 0:
            self.refresh(change['new'][0])

    def refresh(self,item):
        path = os.path.abspath(os.path.join(self.cwd,item))

        if os.path.isfile(path):
            if self.select_file:
                self.accord.set_title(0,path)  
                self.file = path
                self.accord.selected_index = None
            else:
                self.select.value = ()

        else: # os.path.isdir(path)
            self.file = None 
            self.cwd  = path

            # Build list of files and dirs
            keys = ['[..]'];
#             keys = ['shp'];
            for item in os.listdir(path):
                if item[0] == '.':
                    continue
                elif os.path.isdir(os.path.join(path,item)):
                    keys.append('['+item+']'); 
                else:
                    keys.append(item); 

            # Sort and create list of output values
            keys.sort(key=str.lower)
            vals = []
            for k in keys:
                if k[0] == '[':
                    vals.append(k[1:-1]) # strip off brackets
                else:
                    vals.append(k)

            # Update widget
            self.accord.set_title(0,path)  
            self.select.options = list(zip(keys,vals)) 
            with self.select.hold_trait_notifications():
                self.select.value = ()

In [ ]:
# f = PathSelector('/home/' + user + '/pysmm_downloads/')
# display(f.accord)
folder_name =  widgets.Text(
    value='/home/' + user + '/pysmm_downloads/',
    placeholder='Enter your folder name ',
#    description='Google Earth Engine asset ID:',
    disabled=False
#    , continuous_update=True
)
ui1 = HBox([Label('Folder to process: '),folder_name])
display(ui1)



In [ ]:
button = widgets.Button(description='Postprocess')
# button, output, function and linkage
outt = widgets.Output()

def on_button_clicked(b):
    with outt:
        clear_output()
        print('Postprocessing the soil moisture maps')
        !Rscript ~/shared/notebooks/sepal_pysmm/scripts/filter_closing_smm_20190808.R "$folder_name.value"

    
button.on_click(on_button_clicked)
# display
widgets.VBox([button,outt])